In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
from pathlib import Path

In [2]:
from bikesharing.ml_logic.data import get_raw_data ,get_weather_data ,get_polygons
from bikesharing.params import *

- Remove duplicates
- Deal with missing values
- Scale the features
- Encode features
- Perform cyclical engineering

In [3]:
dfs = []
meta = {'years':[], 'n_columns':[], 'n_rows':[]}
for year in range(2019,2023,1):
    df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
    cols = [col.strip() for col in df.columns]
    df.columns = cols
    dfs.append(df)
    meta['years'].append(year)
    meta['n_columns'].append(df.shape[1])
    meta['n_rows'].append(df.shape[0])

/tmp/ipykernel_23698/1700516227.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
/tmp/ipykernel_23698/1700516227.py:4: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')
/tmp/ipykernel_23698/1700516227.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../raw_data/MVG_Rad_Fahrten_{year}.csv', sep=';')


In [4]:
len(dfs)

4

In [5]:
for df in dfs:
    print(df.shape)

(753678, 11)
(721752, 11)
(619573, 11)
(709144, 11)


In [6]:
data = pd.concat(dfs, axis=0)

In [7]:
def pre_process_df(df):
  #vstrip column names
  cols = [col.strip() for col in df.columns]
  df.columns = cols

  # remove column 'Row'
  if 'Row' in df.columns:
    df.drop(columns='Row', inplace=True)

  # select relevant columns only
  #df = df[['STARTTIME', 'STARTLAT', 'STARTLON', 'RENTAL_IS_STATION']].copy()
  
  # make string replacements values
  df_obj = df.select_dtypes(include='object')
  df[df_obj.columns] = df_obj.applymap(lambda x: x.strip().replace(',', '.') if isinstance(x, str) else x)

  # handle datetime
  df.STARTTIME = pd.to_datetime(df.STARTTIME)

  # handle numeric columns
  df.replace('NULL', np.NAN, inplace=True)
  df.replace('', np.NAN, inplace=True)
  df[['STARTLAT', 'STARTLON', 'RENTAL_IS_STATION','ENDLAT', 'ENDLON', 'RETURN_IS_STATION']] = df[['STARTLAT', 'STARTLON', 'RENTAL_IS_STATION','ENDLAT', 'ENDLON', 'RETURN_IS_STATION']].astype(np.float32)
  
  return df

In [8]:
data = pre_process_df(data)

KeyboardInterrupt: 

In [15]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """

    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"

    #print(Fore.BLUE + f"\nSave data to BigQuery @ {full_table_name}...:" + Style.RESET_ALL)


    # reset column names
    data.columns = [f'_{col}' if isinstance(col, int) else col for col in data.columns]

    # Load data onto full_table_name
    client = bigquery.Client()

    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)

    # 🎯 HINT for "*** TypeError: expected bytes, int found":
    # After preprocessing the data, your original column names are gone (print it to check),
    # so ensure that your column names are *strings* that start with either
    # a *letter* or an *underscore*, as BQ does not accept anything else

    print(f"✅ Data saved to bigquery, with shape {data.shape}")


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2804147 entries, 0 to 709143
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   STARTTIME            datetime64[ns]
 1   ENDTIME              object        
 2   STARTLAT             float32       
 3   STARTLON             float32       
 4   ENDLAT               float32       
 5   ENDLON               float32       
 6   RENTAL_IS_STATION    float32       
 7   RENTAL_STATION_NAME  object        
 8   RETURN_IS_STATION    float32       
 9   RETURN_STATION_NAME  object        
dtypes: datetime64[ns](1), float32(6), object(3)
memory usage: 171.2+ MB


In [32]:
load_data_to_bq(
        data=data,
        gcp_project=os.environ.get("GCP_PROJECT"),
        bq_dataset=os.environ.get("BQ_DATASET"),
        table='raw_data_mvg',
        truncate=True
    )

✅ Data saved to bigquery, with shape (2804147, 10)


In [6]:
query =f'''
        SELECT *
        FROM `{GCP_PROJECT}.{BQ_DATASET}.raw_data_mvg`
    '''

In [7]:
df = get_raw_data(GCP_PROJECT, query=query, cache_path=Path(f'{LOCAL_DATA_PATH}/raw/MVG_Rad_Fahrten_{START_YEAR}_to_{END_YEAR}.csv'))
df


Load data from local CSV...


/home/jonathan/code/shoefer987/bike_sharing_demand/bikesharing/ml_logic/data.py:43: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cache_path, header='infer' if data_has_header else None)


✅ Data loaded, with shape (2804147, 10)


,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME
0,2019-01-01 02:47:00,2019-01-01 02:54,48.088402,11.48060,48.087601,11.46165,1.0,Fürstenried West,1.0,Am Sportpark Neuried
1,2019-01-01 04:00:00,2019-01-01 04:22,48.105709,11.41446,48.092430,11.45626,1.0,Bahnhof Ost Planegg,1.0,Kraillinger Weg Neuried
2,2019-01-01 15:29:00,2019-01-01 15:32,48.155258,11.54012,48.152908,11.53310,1.0,Albrechtstraße,1.0,Rotkreuzplatz
3,2019-01-01 15:32:00,2019-01-01 15:40,48.179588,11.58906,48.161991,11.58654,1.0,Anni-Albers-Straße,1.0,Münchner Freiheit
4,2019-01-01 15:45:00,2019-01-01 15:58,48.155231,11.57548,48.139400,11.58888,1.0,Nordendstraße,1.0,Lehel
...,...,...,...,...,...,...,...,...,...,...
2804142,2022-12-17 19:39:00,2022-12-17 19:42,48.161991,11.58654,48.161251,11.59206,1.0,Münchner Freiheit,0.0,NaN
2804143,2022-12-20 19:40:00,2022-12-20 19:48,48.161991,11.58654,48.162029,11.56343,1.0,Münchner Freiheit,0.0,NaN
2804144,2022-12-22 17:03:00,2022-12-22 17:15,48.161991,11.58654,48.180382,11.59943,1.0,Münchner Freiheit,0.0,NaN
2804145,2022-12-28 09:42:00,2022-12-28 09:58,48.161991,11.58654,48.162739,11.57268,1.0,Münchner Freiheit,0.0,NaN


In [8]:
import requests

base_url = 'https://archive-api.open-meteo.com/v1/archive'

params = {
    'latitude': 48.70,
    'longitude': 13.46,
    'start_date' : f'{START_YEAR}-01-01',
    'end_date' : f'{END_YEAR}-12-31',
    'hourly': ['temperature_2m', 'relativehumidity_2m', 'apparent_temperature','windspeed_10m','precipitation']
}

historical_weather_data = requests.get(base_url , params=params).json()

In [4]:
df = get_weather_data(cache_path=Path(f'{LOCAL_DATA_PATH}/raw/Histotical_Weather_Data_{START_YEAR}_to_{END_YEAR}.csv'))
df


Load data from local CSV...
✅ Data loaded, with shape (35064, 6)


,time,temperature_2m,relativehumidity_2m,apparent_temperature,windspeed_10m,precipitation
0,2019-01-01T00:00,3.3,100,0.5,9.0,0.2
1,2019-01-01T01:00,3.4,99,0.4,9.7,0.1
2,2019-01-01T02:00,3.5,100,0.2,12.0,0.2
3,2019-01-01T03:00,3.5,99,0.0,13.5,0.1
4,2019-01-01T04:00,3.5,100,-0.0,14.1,0.0
...,...,...,...,...,...,...
35059,2022-12-31T19:00,6.5,83,3.9,8.0,0.0
35060,2022-12-31T20:00,5.9,83,3.4,6.8,0.0
35061,2022-12-31T21:00,5.8,81,3.1,7.2,0.0
35062,2022-12-31T22:00,6.1,78,3.1,8.8,0.0


In [3]:
dict = get_polygons()
dict

{'Maxvorstadt': <POLYGON ((11.539 48.143, 11.54 48.143, 11.542 48.143, 11.544 48.142, 11.545...>,
 'Schwabing-West': <POLYGON ((11.55 48.161, 11.55 48.161, 11.551 48.16, 11.551 48.16, 11.552 48...>,
 'Au - Haidhausen': <POLYGON ((11.569 48.122, 11.569 48.122, 11.569 48.122, 11.57 48.122, 11.57 ...>,
 'Sendling': <POLYGON ((11.535 48.13, 11.536 48.13, 11.536 48.13, 11.536 48.129, 11.536 4...>,
 'Schwanthalerhöhe': <POLYGON ((11.526 48.137, 11.526 48.137, 11.526 48.136, 11.526 48.136, 11.52...>,
 'Moosach': <POLYGON ((11.466 48.205, 11.466 48.204, 11.467 48.204, 11.468 48.204, 11.46...>,
 'Berg am Laim': <POLYGON ((11.601 48.124, 11.602 48.124, 11.604 48.123, 11.606 48.121, 11.60...>,
 'Trudering': <POLYGON ((11.644 48.115, 11.644 48.115, 11.646 48.114, 11.647 48.113, 11.64...>,
 'Ramersdorf': <POLYGON ((7.14 50.718, 7.143 50.718, 7.145 50.717, 7.147 50.717, 7.147 50.7...>,
 'Obergiesing': <POLYGON ((11.574 48.112, 11.574 48.112, 11.574 48.112, 11.574 48.112, 11.57...>,
 'Untergiesing': 